## Setting up Environment

In [ ]:
### Dependencies
#
# pip install vizdoom + adding gitrepo to folder
# pip install gym

In [2]:
# Imports

from vizdoom import *
import random
import time
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box
import cv2

In [3]:
# Vizdoom OpenAI Gym Environment

from cv2 import resize


class VizDoomGym(Env):

    def __init__(self, render=False):
        # Inherit from Env
        super().__init__()
        # Setup Game
        self.game = DoomGame()
        self.game.load_config("VizDoom/scenarios/basic.cfg")

        # check if render is active or not for faster compute
        if render == False:
            self.game.set_window_visibile(False)
        else:
            self.game.set_window_visible(True)

        self.game.init()

        # Create observation and action space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)
    
    # define a step in the environment
    def step(self, action):
        actions = np.identity(3, dtype=np.uint8)
        reward = self.game.make_action(actions[action],4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        done = self.game.is_episode_finished()

        return state, reward, done, info

    # define how to render the game 
    def render():
        pass

    # define what happens at game start
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state

    # close game 
    def close(self):
        self.game.close()

In [14]:
env = VizDoomGym()

In [15]:
env.close()